<a href="https://colab.research.google.com/github/jungwoo1208/AI_Study/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#문자 단위

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print('문자 집합 사이즈 :{}'.format(vocab_size))
print ('문자 집합 : ', char_vocab)

문자 집합 사이즈 :5
문자 집합 :  ['!', 'a', 'e', 'l', 'p']


In [ ]:
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [ ]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [ ]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [ ]:
x_data = [x_data]
y_data = [y_data]

In [ ]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

In [ ]:
x=torch.FloatTensor(x_one_hot)
y=torch.LongTensor(y_data)

In [ ]:
class Net(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size, bias=True)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

In [ ]:
net = Net(input_size, hidden_size, output_size)

In [ ]:
outputs = net(x)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(x)
    loss = criterion(outputs.view(-1, vocab_size), y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_vocab[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.612802505493164 prediction:  [[4 4 4 1 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppap
1 loss:  1.377203106880188 prediction:  [[4 4 4 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppp!!
2 loss:  1.151977300643921 prediction:  [[4 4 4 3 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppl!
3 loss:  0.948016345500946 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
4 loss:  0.7293846607208252 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
5 loss:  0.5560047030448914 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.4022025465965271 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.2791253328323364 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.2001728117465973 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.1359129101037979 prediction:  [[4 4 3 2 0]] true

# 더 많은 데이터 학습


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


In [59]:
sentence =("동해물과 백두산이 마르고 닳도록"
"하느님이 보우하사 우리나라 만세"
"무궁화 삼천리 화려강산"
"대한 사람 대한으로 길이 보전하세")

In [60]:
char_set= list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [61]:
print(char_dic)

{'려': 0, '느': 1, '궁': 2, '나': 3, '무': 4, '르': 5, '물': 6, '람': 7, '산': 8, '리': 9, '도': 10, '강': 11, '우': 12, '닳': 13, '마': 14, '대': 15, '화': 16, ' ': 17, '으': 18, '님': 19, '이': 20, '라': 21, '동': 22, '만': 23, '백': 24, '고': 25, '해': 26, '하': 27, '천': 28, '록': 29, '로': 30, '길': 31, '두': 32, '보': 33, '삼': 34, '세': 35, '과': 36, '한': 37, '전': 38, '사': 39}


In [62]:
dic_size = len(char_dic)
hidden_size = dic_size
squence_length = 10
learning_rate = 0.1

In [63]:
x_data=[]
y_data=[]

for i in range(0, len(sentence)-squence_length):
    x_str = sentence[i:i+squence_length]
    y_str = sentence[i+1:i+squence_length+1]
    print(i, x_str, '->',y_str)
    x_data.append([char_dic[k] for k in x_str])
    y_data.append([char_dic[k] for k in y_str])

0 동해물과 백두산이  -> 해물과 백두산이 마
1 해물과 백두산이 마 -> 물과 백두산이 마르
2 물과 백두산이 마르 -> 과 백두산이 마르고
3 과 백두산이 마르고 ->  백두산이 마르고 
4  백두산이 마르고  -> 백두산이 마르고 닳
5 백두산이 마르고 닳 -> 두산이 마르고 닳도
6 두산이 마르고 닳도 -> 산이 마르고 닳도록
7 산이 마르고 닳도록 -> 이 마르고 닳도록하
8 이 마르고 닳도록하 ->  마르고 닳도록하느
9  마르고 닳도록하느 -> 마르고 닳도록하느님
10 마르고 닳도록하느님 -> 르고 닳도록하느님이
11 르고 닳도록하느님이 -> 고 닳도록하느님이 
12 고 닳도록하느님이  ->  닳도록하느님이 보
13  닳도록하느님이 보 -> 닳도록하느님이 보우
14 닳도록하느님이 보우 -> 도록하느님이 보우하
15 도록하느님이 보우하 -> 록하느님이 보우하사
16 록하느님이 보우하사 -> 하느님이 보우하사 
17 하느님이 보우하사  -> 느님이 보우하사 우
18 느님이 보우하사 우 -> 님이 보우하사 우리
19 님이 보우하사 우리 -> 이 보우하사 우리나
20 이 보우하사 우리나 ->  보우하사 우리나라
21  보우하사 우리나라 -> 보우하사 우리나라 
22 보우하사 우리나라  -> 우하사 우리나라 만
23 우하사 우리나라 만 -> 하사 우리나라 만세
24 하사 우리나라 만세 -> 사 우리나라 만세무
25 사 우리나라 만세무 ->  우리나라 만세무궁
26  우리나라 만세무궁 -> 우리나라 만세무궁화
27 우리나라 만세무궁화 -> 리나라 만세무궁화 
28 리나라 만세무궁화  -> 나라 만세무궁화 삼
29 나라 만세무궁화 삼 -> 라 만세무궁화 삼천
30 라 만세무궁화 삼천 ->  만세무궁화 삼천리
31  만세무궁화 삼천리 -> 만세무궁화 삼천리 
32 만세무궁화 삼천리  -> 세무궁화 삼천리 화
33 세무궁화 삼천리 화 -> 무궁화 삼천리 화려
34 무궁화 삼천리 화려 -> 궁화 삼천리 화려강
35 궁화 삼천리 화려강 -> 화 삼천리 화려강산
36

In [64]:
print(x_data[0])
print(y_data[0])

[22, 26, 6, 36, 17, 24, 32, 8, 20, 17]
[26, 6, 36, 17, 24, 32, 8, 20, 17, 14]


In [65]:
x_one_hot = [np.eye(dic_size)[x]for x in x_data]
x= torch.FloatTensor(x_one_hot)
y= torch.LongTensor(y_data)

In [66]:
class Net(nn.Module):
    def __init__(self,input_dim,hidden_dim,layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

In [67]:
net = Net(dic_size, hidden_size, 2)

In [68]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [69]:
print(x.shape)
print(y.shape)

torch.Size([54, 10, 40])
torch.Size([54, 10])


In [70]:
outputs=net(x)
print(outputs.shape)

torch.Size([54, 10, 40])


In [71]:
for i in range(100):
  optimizer.zero_grad()
  outputs = net(x)
  # Reshape outputs to (batch_size * sequence_length, dic_size) before calculating loss
  loss = criterion(outputs.view(-1, dic_size), y.view(-1))
  loss.backward()
  optimizer.step()

  # Get predicted indices for each character
  results = outputs.argmax(dim=2)
  # Reshape to (batch_size, sequence_length)
  results = results.view(-1, squence_length)

  predict_str = ""
  for j, result in enumerate(results):
    if j == 0:  # For the first sequence, add all predicted characters
      predict_str += ''.join([char_set[t] for t in result])
    else:  # For subsequent sequences, add only the last predicted character
      predict_str += char_set[result[-1]]

  print(predict_str)  # Print the predicted string

우라라라라라천라라라라라천라라천천라라라라라라라라천라라라라라라라라라라라라라라라라라라라천라천라천라라라천라라라천라라라라라
             리   리     리사리 리       리           리 리     리     리사
님하  전사 전동   사사    사  사  사사  사사  사사 사사   사   사  사사 사 사    사사 사사 
이 이천대 이록사로 대로로  세 로 고로  로   고  로 무로 로    로로   로 로 로로  화    사 마 
  이 마이님이 마님마마마마님사마마마마마마마마화마님마화고마마마마화마마마마화마화마화마마님마화마마마마마마마마마마님마마
 화        화 한   한   화                     화              화  화  
 느 리  록 람 세  록로리   세  느  느 리  느   로산 하  느  느   람 산 로    로리 로느  
       하으만천   르우하궁  하보으 궁 삼   닳만르만궁  보천하나만르   보닳보려  르 한 만르록보   
  삼천만세천보  르닳 만르 님  세천만르 보닳보  만 무르무궁  보르사  르무르닳  만  보 천만르무세천만   
 사이 사람닳사 사 님  세무하 으 사사 르  사  사 사세하 화 하람하 사람무세   보     하    사 도 
무님이 하이 사 사이 사 님 하이   사 사  사이 사  님 사  사 사 하도사이 하 사  사 하하 하이 사 하 
무님이 하이 사 마이 사 님 하  이 마이하이 마이 하 하님 마  사 하 화 로이 사 마이 사  로 하이 마이 사
무궁이 마이하한 마이  만도록하 강이 삼느하   삼 하 마한 한화 삼람리 화 마이 한 삼느 한한 삼느하이 삼느하사
무물이느강이   강이  닳도록이무님  보우하느화 삼 리 마려하 화 마우리 화려만이화  보화  화려만람하   닳하 
무물과려강세 이 만르 이닳과화 이 이 보르님이 로르 이 만세 리느 보닳님 닳려대느님이 님이록이느 이 만이 강르님이
무하과으만우 이 보르  닳도화 이 이 보우하대 로  이 보우무   대천리